## Assignement 2: Exploritory Data Analysis

William "Taylor" Martinez

#### Notes and Setup

[`dplyr` to `polars`](https://docs.pola.rs/user-guide/migration/pandas/#column-assignment):
| Operation                | Syntax                                   |
|--------------------------|------------------------------------------|
| read (lazy)              | `df.scan_csv()` or `df.scan_parquet()`  |
| collect                  | `df.collect()`                           |
| select                   | `df.select("col_name1", "col_name2")`   |
| filter                   | `df.filter(pl.col("col_name") < 10)`    |
| mutate                   | `df.with_columns(new_col_name = pl.col("col_name") * 10)` |
| mutate (conditional)   | ```df.with_columns( pl.when(pl.col("c") == 2) .then(pl.col("b")) .otherwise(pl.col("a")).alias("a") )``` |
| missing                  | `null`                                   |
| all columns               | `pl.all()`                                  |

`csv` vs `parquet`:
    Parquet was chosen over `csv` because it takes up less space, it is columnar formatted, and is has improved query performance. [medium](https://medium.com/@dinesh1.chopra/unveiling-the-battle-apache-parquet-vs-csv-exploring-the-pros-and-cons-of-data-formats-b6bfd8e43107)








In [1]:
import polars as pl # Used for Lazy, Parallel Queries 
import pandas as pd # Compatibility
from great_tables import GT, md, html # Table Generation
import sidetable # Table Generation

from selenium import webdriver

driver = webdriver.Chrome()

data_path = "../../Data/" # Relative Path to Data Folder.
figures_path = "./figures_taylor/" # Relative Path to Figures.

ValueError: Timeout value connect was <object object at 0x100f607d0>, but it must be an int, float or None.

In [ ]:
df = pl.scan_csv(data_path + "heart_2022_with_nans.csv")
df.collect().write_parquet(data_path + "heart_2022_with_nans.parquet")
df = pl.scan_parquet(data_path + "heart_2022_with_nans.parquet")

### EDA Task 1: Create `HadHeartDisease` column

1. Set `HadHeartDisease` to `True` if the survey participant reported having a least one of the following adverse cardiovascular events: heart attack (`HadHeartAttack`), stroke (`HadStroke`), or angina  (`HadAngina`).

In [ ]:
### Create HadHeartDisease Column
df = df.with_columns(
    pl.when(
        (pl.col("HadHeartAttack") == "Yes") |
        (pl.col("HadStroke") == "Yes") |
        (pl.col("HadAngina") == "Yes")
    )
    .then(pl.lit("Yes"))
    .otherwise(pl.lit("No"))
    .alias("HadHeartDisease")
)

df.fetch(5)

State,Sex,GeneralHealth,PhysicalHealthDays,MentalHealthDays,LastCheckupTime,PhysicalActivities,SleepHours,RemovedTeeth,HadHeartAttack,HadAngina,HadStroke,HadAsthma,HadSkinCancer,HadCOPD,HadDepressiveDisorder,HadKidneyDisease,HadArthritis,HadDiabetes,DeafOrHardOfHearing,BlindOrVisionDifficulty,DifficultyConcentrating,DifficultyWalking,DifficultyDressingBathing,DifficultyErrands,SmokerStatus,ECigaretteUsage,ChestScan,RaceEthnicityCategory,AgeCategory,HeightInMeters,WeightInKilograms,BMI,AlcoholDrinkers,HIVTesting,FluVaxLast12,PneumoVaxEver,TetanusLast10Tdap,HighRiskLastYear,CovidPos,HadHeartDisease
str,str,str,f64,f64,str,str,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,str,str,str,str,str,str,str,str
"""Alabama""","""Female""","""Very good""",0.0,0.0,"""Within past ye…","""No""",8.0,null,"""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""Yes""","""No""","""No""","""No""","""No""","""No""","""No""","""Never smoked""","""Not at all (ri…","""No""","""White only, No…","""Age 80 or olde…",null,null,null,"""No""","""No""","""Yes""","""No""","""Yes, received …","""No""","""No""","""No"""
"""Alabama""","""Female""","""Excellent""",0.0,0.0,null,"""No""",6.0,null,"""No""","""No""","""No""","""No""","""Yes""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""Never smoked""","""Never used e-c…","""No""","""White only, No…","""Age 80 or olde…",1.6,68.04,26.57,"""No""","""No""","""No""","""No""","""No, did not re…","""No""","""No""","""No"""
"""Alabama""","""Female""","""Very good""",2.0,3.0,"""Within past ye…","""Yes""",5.0,null,"""No""","""No""","""No""","""No""","""Yes""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""Never smoked""","""Never used e-c…","""No""","""White only, No…","""Age 55 to 59""",1.57,63.5,25.61,"""No""","""No""","""No""","""No""",null,"""No""","""Yes""","""No"""
"""Alabama""","""Female""","""Excellent""",0.0,0.0,"""Within past ye…","""Yes""",7.0,null,"""No""","""No""","""No""","""Yes""","""No""","""No""","""No""","""No""","""Yes""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""Current smoker…","""Never used e-c…","""Yes""","""White only, No…",null,1.65,63.5,23.3,"""No""","""No""","""Yes""","""Yes""","""No, did not re…","""No""","""No""","""No"""
"""Alabama""","""Female""","""Fair""",2.0,0.0,"""Within past ye…","""Yes""",9.0,null,"""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""Never smoked""","""Never used e-c…","""Yes""","""White only, No…","""Age 40 to 44""",1.57,53.98,21.77,"""Yes""","""No""","""No""","""Yes""","""No, did not re…","""No""","""No""","""No"""


### EDA Task 2: Drop Observations With Too Many Missing Values

1. Create `df_heart_drop` where participants are dropped if Heart attack (`HadHeartAttack`), stroke (`HadStroke`), or angina  (`HadAngina`) are missing.

2. From `df_heart_drop`, make multiple dataframes that drop survey participants based
on the number of missing responses.

3. Collect the dataframes and return the length of each entry.


In [ ]:
### Drop observations where HadHeartAttack, HadStroke, and HadAngina are missing

df_heart_drop = (df.drop_nulls(
    subset=["HadHeartAttack", "HadStroke", "HadAngina"]
    ))

# Create a dictionary to keep track of all lazy dataframes
df_dict = {"df" : df, "df_heart_drop" : df_heart_drop}

# If number of missing is > threshold, drop the observation.
thresholds = [0, 1, 3, 5, 10, 20, 40]

for threshold in thresholds:
    df_name = f"df_heart_drop_{threshold:02}" # f-string
    # Drop observation if # of null values is greater than threshold and 
    # Add to dictionary
    df_dict[df_name] = df_heart_drop.filter(
        pl.sum_horizontal(pl.all().is_null()) <= threshold
        )
    # Print the number of rows in each dataframe.
    print(df_name + ": " + str(
        df_dict[df_name].select(pl.len()).collect().item()
        ))

df_dict

df_heart_drop_00: 246022
df_heart_drop_01: 331181
df_heart_drop_03: 381718
df_heart_drop_05: 391725
df_heart_drop_10: 410245
df_heart_drop_20: 436507
df_heart_drop_40: 437510


{'df': <LazyFrame [41 cols, {"State": String … "HadHeartDisease": String}] at 0x16459BFD0>,
 'df_heart_drop': <LazyFrame [41 cols, {"State": String … "HadHeartDisease": String}] at 0x1645BEBD0>,
 'df_heart_drop_00': <LazyFrame [41 cols, {"State": String … "HadHeartDisease": String}] at 0x146F4D090>,
 'df_heart_drop_01': <LazyFrame [41 cols, {"State": String … "HadHeartDisease": String}] at 0x164598A90>,
 'df_heart_drop_03': <LazyFrame [41 cols, {"State": String … "HadHeartDisease": String}] at 0x103C29290>,
 'df_heart_drop_05': <LazyFrame [41 cols, {"State": String … "HadHeartDisease": String}] at 0x1645BFD10>,
 'df_heart_drop_10': <LazyFrame [41 cols, {"State": String … "HadHeartDisease": String}] at 0x1055CBF10>,
 'df_heart_drop_20': <LazyFrame [41 cols, {"State": String … "HadHeartDisease": String}] at 0x1645BFE90>,
 'df_heart_drop_40': <LazyFrame [41 cols, {"State": String … "HadHeartDisease": String}] at 0x146AD16D0>}

### EDA Task 3: Impute remaining missing values

1. Show column types

2. Impute float and integer values by median.

3. Impute string values by mode.

Note: This is applied to df_heart_drop, other dataframes can be imputed.


In [ ]:
### Unique data types:
# print(set(df_heart_drop.dtypes))

### Imputation
def impute_df(df):
    df = df.collect() # Collect because iteration is needed.
    for i in range(len(df.columns)):
        col_name = df.columns[i]
        dtype = df.dtypes[i]
        ## Impute string using the mode
        if dtype == pl.Utf8:
            mode_value = df[col_name].mode()
            df = df.with_columns(df[col_name].fill_null(mode_value))
        ## Impute float using the median
        elif dtype == pl.Float64:
            median_value = df[col_name].median()
            df = df.with_columns(df[col_name].fill_null(median_value))
        ## Warning catch: if type is not a string or float.
        else:
            print("Unexpected type:", dtype)
    return df

### Show the number of missing values for each column
# impute_df(df_heart_drop).null_count()
df_dict_imp = dict()
for key, value in df_dict.items():
    df_name = f"{key}_imp"
    df_dict_imp[df_name] = pl.LazyFrame(impute_df(value))

df_dict_imp


{'df_imp': <LazyFrame [41 cols, {"State": String … "HadHeartDisease": String}] at 0x1645BD510>,
 'df_heart_drop_imp': <LazyFrame [41 cols, {"State": String … "HadHeartDisease": String}] at 0x103C2DFD0>,
 'df_heart_drop_00_imp': <LazyFrame [41 cols, {"State": String … "HadHeartDisease": String}] at 0x164598F90>,
 'df_heart_drop_01_imp': <LazyFrame [41 cols, {"State": String … "HadHeartDisease": String}] at 0x105534D10>,
 'df_heart_drop_03_imp': <LazyFrame [41 cols, {"State": String … "HadHeartDisease": String}] at 0x103C59190>,
 'df_heart_drop_05_imp': <LazyFrame [41 cols, {"State": String … "HadHeartDisease": String}] at 0x1475058D0>,
 'df_heart_drop_10_imp': <LazyFrame [41 cols, {"State": String … "HadHeartDisease": String}] at 0x1475057D0>,
 'df_heart_drop_20_imp': <LazyFrame [41 cols, {"State": String … "HadHeartDisease": String}] at 0x107F2E910>,
 'df_heart_drop_40_imp': <LazyFrame [41 cols, {"State": String … "HadHeartDisease": String}] at 0x1475043D0>}

### EDA Task 4: Show tables/graphs representing each task

1. Table comparing the number of observations of `HadHeartAttack`, `HadStroke`, `HadAngina` and the summarized column `HadHeartDisease`.
    a. Provide an example using a rudimentary model (Logit Regression with enet regularization)

2. Table showing the number of observations after setting a missing threshold.
    a. Provide an example using a rudimentary model (Logit Regression with enet regularization)

3. Table comparing using imputation.

In [ ]:
### Pick your poison. Remember these are lazy and will need to be collected.
### If you want to use pandas, collect then use df.to_pandas()
df_dict
### or
df_dict_imp
### both
df_dict_all = {}
df_dict_all.update(df_dict)
df_dict_all.update(df_dict_imp)
df_dict_all

{'df': <LazyFrame [41 cols, {"State": String … "HadHeartDisease": String}] at 0x16459BFD0>,
 'df_heart_drop': <LazyFrame [41 cols, {"State": String … "HadHeartDisease": String}] at 0x1645BEBD0>,
 'df_heart_drop_00': <LazyFrame [41 cols, {"State": String … "HadHeartDisease": String}] at 0x146F4D090>,
 'df_heart_drop_01': <LazyFrame [41 cols, {"State": String … "HadHeartDisease": String}] at 0x164598A90>,
 'df_heart_drop_03': <LazyFrame [41 cols, {"State": String … "HadHeartDisease": String}] at 0x103C29290>,
 'df_heart_drop_05': <LazyFrame [41 cols, {"State": String … "HadHeartDisease": String}] at 0x1645BFD10>,
 'df_heart_drop_10': <LazyFrame [41 cols, {"State": String … "HadHeartDisease": String}] at 0x1055CBF10>,
 'df_heart_drop_20': <LazyFrame [41 cols, {"State": String … "HadHeartDisease": String}] at 0x1645BFE90>,
 'df_heart_drop_40': <LazyFrame [41 cols, {"State": String … "HadHeartDisease": String}] at 0x146AD16D0>,
 'df_imp': <LazyFrame [41 cols, {"State": String … "HadHeartDis

In [ ]:
### Table 1: Comparing `HadHeartAttack`, `HadStroke`, `HadAngina` to `HadHeartDisease`

print(GT.save.__doc__)


tbl_1 = (df.collect().to_pandas().stb.freq(
    ["HadHeartAttack", "HadStroke", "HadAngina", "HadHeartDisease"])
    .drop(columns=['cumulative_count', 'cumulative_percent'])
)

tbl_1["percent"] = tbl_1["percent"] / 100

tbl_event_freq = (GT(tbl_1)
    .tab_header(title="Adverse Cardiovascular Event Frequencies")
    .cols_label(
        count=html("Count"),  # Centering text using HTML style attribute
        percent=html("Percent")
    )
    .cols_align(align="center")
    .fmt_percent(columns="percent")
    .tab_source_note(
        source_note="• Missing values from HadHeartAttack, HadStroke, and HadAngina were dropped."
    )
)
tbl_event_freq.save(file=f"{figures_path}tbl_event_freq.png")




    Save a table as an image file or a PDF document.

    The `save()` method makes it easy to save a table object as an image file. The function produces
    a high-resolution image file or PDF of the table. The output file is create by first taking a
    screenshot of the table using a headless Chrome browser (other browser options are possible if
    Chrome isn't present). The screenshot is then cropped to only include the table element, with
    some additional pixels added around the table (controlled by the `expand=` parameter). Finally,
    the resulting image is saved to the specified file path in the format specified (via the file
    extension).

    Parameters
    ----------
    file
        The name of the file to save the image to. Accepts names ending with .png, .bmp, and other
        image extensions. Also accepts the extension .pdf.
    selector
        The HTML element selector to use to select the table. By default, this is set to "table",
        which selects the 

ValueError: Timeout value connect was <object object at 0x100b1c7d0>, but it must be an int, float or None.

In [ ]:
### WORK IN PROGRESS DID NOT FINISH
thresholds = [0, 1, 3, 5, 10, 20, 40]
df_names = ["df", "df_heart_drop"]

for threshold in thresholds:
    df_names.append(f"df_heart_drop_{threshold:02}")

# print(df_names)

for df_name in df_names:
    globals()[df_name].group_by(
        ['HadHeartAttack', 'HadStroke', 'HadAngina', 'HadHeartDisease']
        ).agg(pl.len().alias('Count'))



['df', 'df_heart_drop', 'df_heart_drop_00', 'df_heart_drop_01', 'df_heart_drop_03', 'df_heart_drop_05', 'df_heart_drop_10', 'df_heart_drop_20', 'df_heart_drop_40']
